# Projeto Prático Perceptron Rosenblatt

Neste projeto prático, o objetivo é implementar o algoritmo de treinamento mediante Aprendizado Supervisionado 
do neurônio Perceptron de Rosenblatt aplicado em problemas de classificação.

## Equipe:

1. Afonso Henrique Torres Lucas | ifonso.developer@gmail.com | iFonso - 2215080047
2. Erik Gustavo Lima de Oliveira | erik.exatas10@gmail.com | ErikExatas - 2115080049
3. David Augusto De Oliveira E Silva | david-augusto-silva - 2115080006
4. Lilian Iazzai De Souza Oliveira | lilianiazzai - 2215080018
5. Vitor Nascimento Aguiar | Vtaguiar1909 - 2115080055

In [1]:
# identificador do arquivo
id = (7+9+6+8+5)%4
print(id)

3


In [2]:
# Bibliotecas
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as skm
from prettytable import PrettyTable

# Módulos
from modules.perceptron import Perceptron

In [3]:
def get_ndarray_from_file(filename: str) -> np.ndarray:
    """Formato dos dados -> (m, 1, 3) : [[[x_1, x_2, y_d]], ...]"""
    return np.squeeze(np.fromfile(filename, dtype=np.float64).reshape(-1, 1, 3))

In [4]:
def split_data(data: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Faz split do dataset -> (m, 2), (m, 1)"""
    return data[:, :2], data[:, 2].reshape(-1, 1)

In [5]:
data = get_ndarray_from_file(f"./data/dataAll.txt")
X, y = split_data(data)

print(f"data: {data.shape} -> {data[0]}")
print(f"X: {X.shape} | y: {y.shape}")

data: (1000, 3) -> [-363.7884  244.1423    0.    ]
X: (1000, 2) | y: (1000, 1)


## Parte I - Problema Linearmente Separável

## Parte II - Experimentação

Nesta parte, as equipes usarão o seu identificador para trabalhar com um arquivo específico. Usaremos, então, o arquivo ```data3.txt```.

Aproveitando o algoritmo construído na parte I, serão executadas 10 repetições deste para seguintes configurações: $\eta \times I = \{0.4, 0.1, 0.01\} \times \{(-100, +100), (-0.5, +0.5)\}$, em que I é o intervalo utilizado para distribuição uniforme do valor dos pesos. No total, serão feitas 60 execuções: 10 execuções para cada uma das 6 configurações.

In [6]:
data = get_ndarray_from_file(f"./data/data{id}.txt")
X3, y3 = split_data(data)

print(f"Tamanho da amosta: {data.shape}")

Tamanho da amosta: (600, 3)


In [7]:
intervals = ((-0.5, 0.5), (-100, 100))
learning_rates = (0.4, 0.1, 0.01)

In [8]:
tabela = PrettyTable([
    "Taxa de aprendizado",
    "Intervalo de Pesos",
    "Quantidade de Ajustes",
    "Menor número de épocas para convergência"
])

for interval in intervals:
    for l in learning_rates:
        epochs = []
        adjustments = []

        for _ in range(10):
            p1 = Perceptron(X3.shape[1], interval)
            e, a = p1.train(X3, y3, l)
            epochs.append(e)
            adjustments.append(a)

        avg = np.average(adjustments)
        std_dev = np.std(adjustments)

        converged_epochs = [ep for ep in epochs if ep != -1]
        min_epochs = min(converged_epochs) if converged_epochs else "N/A"

        tabela.add_row([
            f"{l}",
            f"{interval[0]}, {interval[1]}",
            f"{avg:.1f} ± {std_dev:.1f}",
            f"{min_epochs}"
        ])

In [9]:
tabela

Taxa de aprendizado,Intervalo de Pesos,Quantidade de Ajustes,Menor número de épocas para convergência
0.4,"-0.5, 0.5",784.7 ± 161.3,3
0.1,"-0.5, 0.5",817.6 ± 143.1,5
0.01,"-0.5, 0.5",862.3 ± 204.1,4
0.4,"-100, 100",662.2 ± 506.4,2
0.1,"-100, 100",1281.1 ± 876.2,7
0.01,"-100, 100",10903.8 ± 5522.1,52
